<a href="https://colab.research.google.com/github/shanikairoshi/QFL_Experiments/blob/main/QFL_FAA_Beta_ICOIN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:

%%capture
!pip install genomic-benchmarks
!pip install qiskit qiskit_machine_learning qiskit_algorithms
!pip install qiskit-aer

Teleportation Configuration

In [16]:
from dataclasses import dataclass
import numpy as np

@dataclass
class TeleportCfg:
    use_teleportation: bool = True
    noise: str = "med"     # "low" | "med" | "high"
    shots: int = 256       # samples to estimate fidelity
    alpha: float = 1.0     # fidelity emphasis
    gamma: float = 0.0     # latency penalty exponent
    delta: float = 0.0     # instability penalty exponent
    beta: float = 1.0      # 1=circular, 0=linear blend (only used for angle params)
    seed: int = 2025

_NOISE_P = {"low": 0.02, "med": 0.06, "high": 0.12}

class TeleportationLink:
    """Toy fidelity sampler for a Bell-pair teleportation pipeline."""
    def __init__(self, cfg: TeleportCfg):
        self.cfg = cfg
        self.rng = np.random.default_rng(cfg.seed)
    def sample_fidelity(self, shots=None, noise=None):
        if shots is None: shots = self.cfg.shots
        if noise is None: noise = self.cfg.noise
        p = _NOISE_P.get(noise, 0.06)
        u = self.rng.uniform(0.5, 1.0, size=shots)
        flips = self.rng.binomial(1, p, size=shots)
        return 1.0 - flips * u  # array in [0,1]


In [17]:

from genomic_benchmarks.dataset_getters.pytorch_datasets import DemoHumanOrWorm

test_set = DemoHumanOrWorm(split='test', version=0)
train_set = DemoHumanOrWorm(split='train', version=0)

data_set = train_set
# data_set = train_set + test_set
len(data_set)
from genomic_benchmarks.dataset_getters.pytorch_datasets import DemoHumanOrWorm

test_set = DemoHumanOrWorm(split='test', version=0)
train_set = DemoHumanOrWorm(split='train', version=0)

data_set = train_set
# data_set = train_set + test_set
len(data_set)

from collections import defaultdict
import numpy as np

word_size = 40
word_combinations = defaultdict(int)
iteration = 1
for text, _ in data_set:
    for i in range(len(text)):
        word = text[i:i+word_size]
        if word_combinations.get(word) is None:
          word_combinations[word] = iteration
          iteration += 1



print("First sample int the data_set variable: ")
print(data_set[0])

print("\nFirst 5 samples in the word_combinations dict.")
for key, value in list(word_combinations.items())[:5]:
    print(key, value)


import numpy as np
# Preprocess the training set
np_data_set = []
for i in range(len(data_set)):
    sequence, label = data_set[i]
    sequence = sequence.strip()  # Remove any leading/trailing whitespace
    words = [sequence[i:i + word_size] for i in range(0, len(sequence), word_size)]  # Split the sequence into 4-letter words
    int_sequence = np.array([word_combinations[word] for word in words])
    data_point = {'sequence': int_sequence, 'label': label}
    np_data_set.append(data_point)


print("First 5 samples of encoded data:")
np_data_set[:5]


rng = np.random.default_rng(42)   # fixed seed for reproducibility
rng.shuffle(np_data_set)          # shuffle in place, but reproducible
print("First 5 samples of encoded shuffled data:")
np_data_set[:5]
from sklearn.preprocessing import MinMaxScaler

sequences = np.array([item['sequence'] for item in np_data_set])
sequences = np.vstack(sequences)

scaler = MinMaxScaler()

sequences_scaled = scaler.fit_transform(sequences)

for i, item in enumerate(np_data_set):
    item['sequence'] = sequences_scaled[i]

print("First 5 samples of scaled encoded shuffled data:")
np_data_set[:5]


np_train_data = np_data_set[:70000]
np_test_data = np_data_set[-5000:]

print(f"Length of np_train_data: {len(np_train_data)}")
print(f"Length of np_test_data: {len(np_test_data)}")




np_train_data = np_data_set[:70000]
np_test_data = np_data_set[-5000:]

print(f"Length of np_train_data: {len(np_train_data)}")
print(f"Length of np_test_data: {len(np_test_data)}")

test_sequences = [data_point["sequence"] for data_point in np_test_data]
test_labels = [data_point["label"] for data_point in np_test_data]
test_sequences = np.array(test_sequences)
test_labels = np.array(test_labels)
print(f"Length of Test data: {len(test_sequences)}")


First sample int the data_set variable: 
('TACATCATTGGTTCGGGACGCTGGTGGAAATAGTGAATCCGGCATGTCAGTGAAAGTAAATGAACTTCTTCGATTATACTCGGCAAATGAGAAGTACGGGAATGGAATTATGCTCCTTAACGCCTATAAATCTGTTATTCATAACTTTTCCGGTTTTCCCAAAACCTACCCATTTTTGAGCAAAATTGCCAACGTAGGCA', 0)

First 5 samples in the word_combinations dict.
TACATCATTGGTTCGGGACGCTGGTGGAAATAGTGAATCC 1
ACATCATTGGTTCGGGACGCTGGTGGAAATAGTGAATCCG 2
CATCATTGGTTCGGGACGCTGGTGGAAATAGTGAATCCGG 3
ATCATTGGTTCGGGACGCTGGTGGAAATAGTGAATCCGGC 4
TCATTGGTTCGGGACGCTGGTGGAAATAGTGAATCCGGCA 5
First 5 samples of encoded data:
First 5 samples of encoded shuffled data:
First 5 samples of scaled encoded shuffled data:
Length of np_train_data: 70000
Length of np_test_data: 5000
Length of np_train_data: 70000
Length of np_test_data: 5000
Length of Test data: 5000


In [18]:
# Verify the structure of np_test_data
for i, data_point in enumerate(np_test_data[:5]):  # Print the first 5 test data points
    print(f"Test data point {i}: {data_point}")

Test data point 0: {'sequence': array([0.74898835, 0.74898835, 0.74898835, 0.74898835, 0.74898835]), 'label': 1}
Test data point 1: {'sequence': array([0.270534, 0.270534, 0.270534, 0.270534, 0.270534]), 'label': 0}
Test data point 2: {'sequence': array([0.55422793, 0.55422793, 0.55422793, 0.55422793, 0.55422793]), 'label': 1}
Test data point 3: {'sequence': array([0.52057059, 0.52057059, 0.52057059, 0.52057059, 0.52057059]), 'label': 1}
Test data point 4: {'sequence': array([0.20668716, 0.20668716, 0.20668716, 0.20668716, 0.20668716]), 'label': 0}


In [19]:
import time
from qiskit.circuit.library import ZZFeatureMap, RealAmplitudes
from qiskit_algorithms.optimizers import COBYLA
from qiskit_machine_learning.algorithms.classifiers import VQC
from qiskit.primitives import BackendSampler
from functools import partial
from qiskit_aer import Aer

from qiskit_machine_learning.neural_networks import SamplerQNN
from qiskit_machine_learning.algorithms.classifiers import NeuralNetworkClassifier
from qiskit.circuit.library import ZZFeatureMap, RealAmplitudes
from qiskit.primitives import BackendSampler
from qiskit_algorithms.optimizers import SPSA
import numpy as np
import time
from IPython.display import clear_output
import matplotlib.pyplot as plt

import time
from qiskit.circuit.library import ZZFeatureMap, RealAmplitudes
from qiskit_algorithms.optimizers import COBYLA
from qiskit_machine_learning.algorithms.classifiers import VQC
from qiskit.primitives import BackendSampler
from functools import partial
from qiskit_aer import Aer

from qiskit_machine_learning.neural_networks import SamplerQNN
from qiskit_machine_learning.algorithms.classifiers import NeuralNetworkClassifier
from qiskit.circuit.library import ZZFeatureMap, RealAmplitudes
from qiskit.primitives import BackendSampler
from qiskit_algorithms.optimizers import SPSA
import numpy as np
import time
from IPython.display import clear_output
import matplotlib.pyplot as plt


num_clients = 5
num_epochs = 20
max_train_iterations = 50
samples_per_epoch=100
backend = Aer.get_backend('aer_simulator')

class Client:
   def __init__(self, train_data):  # Add test_data to __init__
        self.client_train_data = train_data
        #self.test_data = test_data  # Store test_data as an attribute
        self.models = []
        self.train_scores = []
        self.test_scores = []
        self.primary_model = None
'''
def split_dataset(num_clients, num_epochs, samples_per_epoch):
  clients = []
  for i in range(num_clients):
    client_data = []
    for j in range(num_epochs):
      start_idx = (i*num_epochs*samples_per_epoch)+(j*samples_per_epoch)
      end_idx = (i*num_epochs*samples_per_epoch)+((j+1)*samples_per_epoch)
      client_data.append(np_train_data[start_idx:end_idx])
    # Pass test_data when creating Client instances
    clients.append(Client(client_data, np_test_data))
  return clients

clients = split_dataset(num_clients, num_epochs, samples_per_epoch)
'''
def split_dataset(num_clients, num_epochs, samples_per_epoch):
    clients = []
    # Split test data across clients
    #test_samples_per_client = len(test_sample_sequences) // num_clients

    for i in range(num_clients):
        client_train_data = []
        for j in range(num_epochs):
            start_idx = (i * num_epochs * samples_per_epoch) + (j * samples_per_epoch)
            end_idx = (i * num_epochs * samples_per_epoch) + ((j + 1) * samples_per_epoch)
            client_train_data.append(np_train_data[start_idx:end_idx])
            #print(f"Client {i+1} training data size: {len(np_train_data[start_idx:end_idx])}")
        # Assign a subset of the test data to each client
        #test_start_idx = i * test_samples_per_client
        #test_end_idx = (i + 1) * test_samples_per_client
        #client_test_data = test_sample_sequences[test_start_idx:test_end_idx]

        # Create Client instance with both train and test data
        clients.append(Client(client_train_data))

    return clients

clients = split_dataset(num_clients, num_epochs, samples_per_epoch)

itr = 0
def training_callback(weights, obj_func_eval):
        global itr
        itr += 1
        print(f"{itr}", end=' | ')

In [20]:
from qiskit_machine_learning.neural_networks import SamplerQNN
from qiskit_machine_learning.algorithms.classifiers import NeuralNetworkClassifier
from qiskit.circuit.library import ZZFeatureMap, RealAmplitudes
from qiskit.primitives import BackendSampler
from qiskit_algorithms.optimizers import SPSA
import numpy as np
import time
from IPython.display import clear_output
import matplotlib.pyplot as plt

# Callback function to visualize training progress
objective_func_vals = []
def callback_graph(weights, obj_func_eval):
    clear_output(wait=True)
    objective_func_vals.append(obj_func_eval)
    plt.title("Objective function value against iteration")
    plt.xlabel("Iteration")
    plt.ylabel("Objective function value")
    plt.plot(range(len(objective_func_vals)), objective_func_vals)
    plt.show()

In [21]:
# Function to initialize a new QNN model (same architecture as clients' models)
def initialize_model(num_features):
    # Create the same quantum neural network (QNN) architecture as clients
    feature_map = ZZFeatureMap(feature_dimension=num_features, reps=2)
    ansatz = RealAmplitudes(num_qubits=num_features, reps=3)

    # Combine the feature map and ansatz into a single quantum circuit
    qc = feature_map.compose(ansatz)

    # Use parity as the interpretation function
    def parity(x):
        return "{:b}".format(x).count("1") % 2  # Binary classification

    # Explicitly define input_params and weight_params
    input_params = feature_map.parameters  # Input parameters (for encoding)
    weight_params = ansatz.parameters      # Trainable parameters (for optimization)

    # Define the QNN model using a sampler
    sampler_qnn = SamplerQNN(
        circuit=qc,
        interpret=parity,
        output_shape=2,  # Binary classification
        input_params=input_params,
        weight_params=weight_params
    )

    # Create a classifier using the neural network
    qnn_classifier = NeuralNetworkClassifier(
        neural_network=sampler_qnn,
        optimizer=SPSA(maxiter=50)  # Use SPSA optimizer
    )

    return qnn_classifier

# Function to create a model with averaged weights
def create_model_with_weights(average_weights, num_features):
    # Initialize a new QNN model with the same architecture
    model = initialize_model(num_features)
    # Assign the averaged weights to the model's trainable parameters (ansatz weights)
    weight_params = model.neural_network.weight_params  # Get the trainable parameters

    # Check if the lengths match, and truncate if necessary
    num_weights = min(len(average_weights), len(weight_params))

    # Create a dictionary mapping parameters to averaged weights
    param_dict = {param: average_weights[i] for i, param in enumerate(weight_params[:num_weights])}


    # Assign the averaged weights to the circuit parameters
    model.neural_network.circuit.assign_parameters(param_dict)

    return model


In [22]:
from qiskit_machine_learning.neural_networks import SamplerQNN
from qiskit_machine_learning.algorithms.classifiers import NeuralNetworkClassifier
from qiskit.circuit.library import ZZFeatureMap, RealAmplitudes
from qiskit.primitives import BackendSampler
from qiskit_algorithms.optimizers import SPSA
import numpy as np
import time
from IPython.display import clear_output
import matplotlib.pyplot as plt

# Callback function to visualize training progress
objective_func_vals = []
def callback_graph(weights, obj_func_eval):
    clear_output(wait=True)
    objective_func_vals.append(obj_func_eval)
    plt.title("Objective function value against iteration")
    plt.xlabel("Iteration")
    plt.ylabel("Objective function value")
    plt.plot(range(len(objective_func_vals)), objective_func_vals)
    plt.show()

# Function to create the QNN model
def create_qnn_model(num_features):
    # num_features = data_train[0]["sequence"].shape[0]  # Remove this line as data_train is not defined
    # Define the quantum feature map and ansatz
    feature_map = ZZFeatureMap(feature_dimension=num_features, reps=2)  # Use num_features passed as argument
    ansatz = RealAmplitudes(num_qubits=num_features, reps=3)

    # Construct the quantum neural network using a sampler
    qc = feature_map.compose(ansatz)  # Build the QNN circuit
    print(f"Number of features (input dimension): {num_features}")
    print(f"Number of circuit parameters: {qc.num_parameters}")

    # Use parity as the interpretation function
    def parity(x):
        return "{:b}".format(x).count("1") % 2  # Binary classification

    # Explicitly define input_params and weight_params
    input_params = feature_map.parameters  # Parameters for the feature map (inputs)
    weight_params = ansatz.parameters      # Parameters for the ansatz (weights)
    #print(f"Input Parameters: {input_params}")
    #print(f"Weight Parameters: {weight_params}")

    sampler_qnn = SamplerQNN(
        circuit=qc,
        interpret=parity,
        output_shape=2,  # Output dimension for binary classification
        input_params=input_params,       # Pass input parameters
        weight_params=weight_params     # Pass weight parameters
    )

    # Define a classifier using the QNN
    qnn_classifier = NeuralNetworkClassifier(
        neural_network=sampler_qnn,
        optimizer=SPSA(maxiter=50),  # Example with SPSA optimizer
        #callback=callback_graph
    )

    return qnn_classifier

In [23]:
def getAccuracy(weights, num_features, test_sequences, test_labels):
    # Rebuild the QNN model with the given weights
    feature_map = ZZFeatureMap(feature_dimension=num_features, reps=1)
    ansatz = RealAmplitudes(num_qubits=num_features, reps=3)

    # Replace bind_parameters with assign_parameters
    # Create a parameter dictionary for assignment
    param_dict = {param: weight for param, weight in zip(ansatz.parameters, weights)}
    ansatz = ansatz.assign_parameters(param_dict)

    # Rebuild the QNN using the updated ansatz
    qc = feature_map.compose(ansatz)

    # Define the parity function for binary classification
    def parity(x):
        return "{:b}".format(x).count("1") % 2

    # Build the SamplerQNN with the updated circuit
    sampler_qnn = SamplerQNN(
        circuit=qc,
        interpret=parity,
        output_shape=2,  # Binary classification
        input_params=feature_map.parameters,  # Input parameters (from the feature map)
        weight_params=ansatz.parameters  # Weight parameters (from the ansatz)
    )

    # Build the NeuralNetworkClassifier with the QNN
    qnn_classifier = NeuralNetworkClassifier(
        neural_network=sampler_qnn,
        optimizer=COBYLA(maxiter=0)  # No need for further optimization
    )

    # Train the classifier on a subset of test data (or use full test data if preferred)
    qnn_classifier.fit(test_sequences, test_labels)

    # Return the accuracy on a larger test set
    return qnn_classifier.score(test_sequences, test_labels)


In [24]:
import os
from google.colab import drive
drive.mount('/content/drive')

CSV_PATH = '/content/drive/MyDrive/ICOIN_2025/federated_qnn_metrics_genome.csv'  # change if you like
os.makedirs(os.path.dirname(CSV_PATH), exist_ok=True)

import csv
import numpy as np

def init_metrics_csv(csv_path: str, num_clients: int):
    """Create/overwrite CSV with a header."""
    header = ['Epoch', 'GlobalAccuracy']
    for i in range(num_clients):
        header += [f'Client{i}_TrainAcc', f'Client{i}_TestAcc']
    with open(csv_path, 'w', newline='') as f:
        csv.writer(f).writerow(header)

def append_metrics_row(csv_path: str, epoch: int, global_acc: float,
                       train_accs: list[float], test_accs: list[float]):
    """Append one epoch’s metrics."""
    row = [epoch, global_acc] + [
        x for pair in zip(train_accs, test_accs) for x in pair
    ]
    with open(csv_path, 'a', newline='') as f:
        csv.writer(f).writerow(row)

def compute_global_accuracy(model, clients: list[dict]) -> float:
    """Evaluate the current global model on the union of all client test sets."""
    X = np.concatenate([c["test_data"]["sequence"] for c in clients], axis=0)
    y = np.concatenate([c["test_data"]["label"]   for c in clients],   axis=0)
    return float(model.score(X, y))


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Trust-weighted angle aggregation (drop-in function)

In [25]:
'''
def agg_faaa_beta_with_qos(
    local_ws,             # List[np.ndarray], per-client parameter vectors
    shard_sizes,          # List[int], data counts per client (|D_n|)
    fidelities,           # np.ndarray shape (K,)
    latencies,            # np.ndarray shape (K,)
    instabilities,        # np.ndarray shape (K,)
    tele_cfg: TeleportCfg,
    angle_mask=None,      # optional boolean mask for angular params
):
    # base client weights ~ data size
    Wn = np.asarray(shard_sizes, float)
    Wn = Wn / (Wn.sum() + 1e-12)

    F = np.clip(np.asarray(fidelities, float), 0.0, 1.0)
    T = np.maximum(np.asarray(latencies, float), 1e-6)
    V = np.maximum(np.asarray(instabilities, float), 0.0)

    trust = (F ** tele_cfg.alpha) / ((T ** tele_cfg.gamma) * ((V + 1e-8) ** tele_cfg.delta))
    W = trust * Wn
    W = W / (W.sum() + 1e-12)

    # Temporarily rescale shard_sizes by trust for your existing aggregator
    shard_sizes_eff = (W * 1000).astype(int).tolist()  # only ratios matter

    # Use your angle-aware aggregator (‘auto’ chooses circular vs linear)
    avg_w, diag = aggregate_weights_angle_aware(
        local_ws, shard_sizes=shard_sizes_eff, mode="auto", angle_mask=angle_mask
    )

    # Optional β-blend: if you want to force a circular/linear blend explicitly,
    # set tele_cfg.beta in aggregate_weights_angle_aware (not needed if mode='auto').
    diag["trust_weights_mean"] = float(W.mean())
    return avg_w, diag
'''
def agg_faaa_beta_with_qos(
    local_ws, shard_sizes, fidelities, latencies, instabilities, tele_cfg, angle_mask=None
):
    Wn = np.asarray(shard_sizes, float); Wn /= (Wn.sum() + 1e-12)
    F = np.clip(np.asarray(fidelities, float), 0.0, 1.0)
    T = np.maximum(np.asarray(latencies, float), 1e-6)
    V = np.maximum(np.asarray(instabilities, float), 0.0)

    trust = (F ** tele_cfg.alpha) / ((T ** tele_cfg.gamma) * ((V + 1e-8) ** tele_cfg.delta))
    W = trust * Wn
    W = W / (W.sum() + 1e-12)  # normalized trust weights actually applied

    # use W as effective shard sizes for angle-aware aggregation
    shard_sizes_eff = (W * 1000).astype(int).tolist()
    avg_w, diag = aggregate_weights_angle_aware(local_ws, shard_sizes=shard_sizes_eff,
                                                mode="auto", angle_mask=angle_mask)
    diag["used_mode"] = diag.get("used_mode", "auto")
    return avg_w, diag, W  # <-- return W


tele_cfg = TeleportCfg(use_teleportation=True, noise="med", shots=256,
                       alpha=1.0, gamma=0.0, delta=0.0, beta=1.0, seed=2025)
tele_link = TeleportationLink(tele_cfg)

def measure_instability(weights_vec: np.ndarray) -> float:
    return float(np.var(weights_vec))

def extract_client_weights(model) -> np.ndarray:
    # Works for NeuralNetworkClassifier(SamplerQNN) and VQC: get the *trainable* weights
    if hasattr(model, "weights") and model.weights is not None:
        return np.asarray(model.weights, float)
    # Fallback: try ansatz parameters (bound values) if available
    params = model.neural_network.weight_params if hasattr(model, "neural_network") else []
    # If you can’t read values directly, keep the model’s own .weights path
    return np.asarray([float(p._symbol_expr) if hasattr(p, "_symbol_expr") else 0.0 for p in params], float)


In [26]:
import csv
from pathlib import Path

def append_round_telemetry(path, epoch, global_acc, fidelities, latencies, instabilities, weights, diag):
    path = Path(path); path.parent.mkdir(parents=True, exist_ok=True)
    header = ["epoch","global_acc","fid_mean","fid_std","lat_mean","lat_p90","instab_mean",
              "weight_entropy","used_mode","R_mean","straddle_frac","sse_geo_gap"]
    write_header = not path.exists()
    with open(path, "a", newline="") as f:
        w = csv.writer(f)
        if write_header: w.writerow(header)
        # simple weight entropy (higher ⇒ more uniform)
        eps = 1e-12
        ent = float(-np.sum(weights * np.log(weights + eps)))
        row = [
            int(epoch), float(global_acc),
            float(np.mean(fidelities)), float(np.std(fidelities)),
            float(np.mean(latencies)), float(np.percentile(latencies,90)),
            float(np.mean(instabilities)),
            ent,
            str(diag.get("used_mode","auto")), float(diag["R_mean"]), float(diag["straddle_frac"]),
            float(diag["sse_geo_gap"]),
        ]
        w.writerow(row)

def append_client_telemetry(path, epoch, per_client_rows):
    path = Path(path); path.parent.mkdir(parents=True, exist_ok=True)
    header = ["epoch","client_id","shard_size",
              "train_acc","test_acc","train_loss","test_loss",
              "latency_sec","fidelity_mean","instability_var","trust_weight"]
    write_header = not path.exists()
    with open(path, "a", newline="") as f:
        w = csv.writer(f)
        if write_header: w.writerow(header)
        for r in per_client_rows:
            w.writerow([
                int(epoch), int(r["client_id"]), int(r["shard_size"]),
                float(r["train_acc_local"]), float(r["test_acc_local"]),
                float(r["train_loss_local"]), float(r["test_loss_local"]),
                float(r["time_sec_local"]), float(r["fidelity_mean"]),
                float(r["instability_var"]), float(r["trust_weight"]),
            ])


In [27]:
import numpy as np
from typing import List, Optional, Tuple, Dict

# ---------- Angle utilities ----------
def _wrap_to_pi(x: np.ndarray) -> np.ndarray:
    """Map real numbers to (-π, π]."""
    return (x + np.pi) % (2*np.pi) - np.pi

def _unwrap_to_ref(A: np.ndarray) -> np.ndarray:
    """
    Unwrap rows to be closest to the first row (reference).
    A: (K, D) angles in (-π, π].
    """
    K, D = A.shape
    out = A.copy()
    ref = out[0]
    for i in range(1, K):
        delta = out[i] - ref
        out[i] = ref + ((delta + np.pi) % (2*np.pi) - np.pi)
    return out

def _weighted_circular_mean(A: np.ndarray, W: np.ndarray) -> np.ndarray:
    """
    Coordinate-wise weighted circular mean.
    A: (K, D) in (-π, π]; W: (K,), sum=1.
    """
    C = np.sum(W[:, None] * np.cos(A), axis=0)
    S = np.sum(W[:, None] * np.sin(A), axis=0)
    return np.arctan2(S, C)

def _geodesic_sse(angles: np.ndarray, center: np.ndarray, W: np.ndarray) -> float:
    """Weighted sum of squared geodesic distances (torus)."""
    diff = _wrap_to_pi(angles - center)    # (K, D)
    sse_per_client = np.sum(diff**2, axis=1)
    return float(np.sum(W * sse_per_client))

def _min_covering_arc_length(angles_1d: np.ndarray) -> float:
    """Minimal arc length covering all angles in [0, 2π] metric."""
    a = np.sort(angles_1d)
    gaps = np.diff(a, append=a[0] + 2*np.pi)
    return float(2*np.pi - np.max(gaps))

def angle_diagnostics(local_ws: List[np.ndarray], shard_sizes: List[int]) -> Dict[str, object]:
    """
    Interpret local_ws as angles in (-π, π]; return summary stats for auto-pick.
    """
    A = np.stack(local_ws, axis=0)                 # (K, D)
    W = np.asarray(shard_sizes, float)
    W /= W.sum()

    C = np.sum(W[:, None] * np.cos(A), axis=0)
    S = np.sum(W[:, None] * np.sin(A), axis=0)
    R = np.sqrt(C**2 + S**2)
    R_mean = float(np.mean(R))
    R_min  = float(np.min(R))

    mu_circ = np.arctan2(S, C)
    A_unwrap = _unwrap_to_ref(A)
    mu_lin_unwrapped = np.sum(W[:, None] * A_unwrap, axis=0)
    mu_lin = _wrap_to_pi(mu_lin_unwrapped)

    sse_circ = _geodesic_sse(A, mu_circ, W)
    sse_lin  = _geodesic_sse(A, mu_lin,  W)
    sse_gap  = float(sse_lin - sse_circ)  # > 0 ⇒ circular better intrinsically

    cover = np.array([_min_covering_arc_length(A[:, j]) for j in range(A.shape[1])])
    straddle_frac = float(1.0 - np.mean(cover <= np.pi))

    return {
        "R_mean": R_mean,
        "R_min": R_min,
        "straddle_frac": straddle_frac,
        "sse_geo_gap": sse_gap,
        "mu_circ": mu_circ,
        "mu_lin":  mu_lin,
    }

# ---------- Aggregators ----------
def fedavg_linear(local_ws: List[np.ndarray], shard_sizes: List[int]) -> np.ndarray:
    W = np.asarray(shard_sizes, float); W /= W.sum()
    A = np.stack(local_ws, axis=0)                 # (K, D)
    return np.sum(W[:, None] * A, axis=0)

def fedavg_circular(local_ws: List[np.ndarray], shard_sizes: List[int]) -> np.ndarray:
    W = np.asarray(shard_sizes, float); W /= W.sum()
    A = np.stack(local_ws, axis=0)                 # (K, D) angles
    return _weighted_circular_mean(A, W)

def _autopick_mode(diag: Dict[str, object],
                   r_mean_thresh: float = 0.85,
                   straddle_thresh: float = 0.05) -> str:
    """
    Rule: prefer circular if geodesic SSE gap > 0, or straddling occurs, or concentration is low.
    """
    if (diag["sse_geo_gap"] > 0.0) or (diag["straddle_frac"] > straddle_thresh) or (diag["R_mean"] < r_mean_thresh):
        return "circular"
    return "linear"

def aggregate_weights_angle_aware(
    local_ws: List[np.ndarray],
    shard_sizes: List[int],
    mode: str = "auto",
    angle_mask: Optional[np.ndarray] = None
) -> Tuple[np.ndarray, Dict[str, object]]:
    """
    Angle-aware aggregation on a mixed parameter vector.
    - local_ws: list of client vectors (length D)
    - shard_sizes: client weights (e.g., data counts)
    - mode ∈ {'linear','circular','auto'}
    - angle_mask: boolean mask of length D; True ⇒ treat as angle in (-π, π]
    Returns: (avg_weights, diagnostics)
    """
    A = np.stack(local_ws, axis=0)                 # (K, D)
    D = A.shape[1]
    W = np.asarray(shard_sizes, float); W /= W.sum()

    if angle_mask is None:
        # If you only have circuit angles in 'weights', set all True.
        angle_mask = np.ones(D, dtype=bool)

    ang_idx = angle_mask
    non_idx = ~angle_mask

    diag = {"used_mode": None}
    if np.any(ang_idx):
        diag_angles = angle_diagnostics([w[ang_idx] for w in local_ws], shard_sizes)
    else:
        # No angular coordinates ⇒ reduce to linear FedAvg
        return fedavg_linear(local_ws, shard_sizes), {"used_mode": "linear"}

    pick = mode if mode != "auto" else _autopick_mode(diag_angles)

    out = np.zeros(D, dtype=float)
    if pick == "linear":
        out[:] = fedavg_linear(local_ws, shard_sizes)
    elif pick == "circular":
        circ = fedavg_circular([w[ang_idx] for w in local_ws], shard_sizes)
        out[ang_idx] = _wrap_to_pi(circ)
        if np.any(non_idx):
            out[non_idx] = fedavg_linear([w[non_idx] for w in local_ws], shard_sizes)
    else:
        raise ValueError("mode must be one of {'linear','circular','auto'}")

    diag.update(diag_angles)
    diag["used_mode"] = pick
    return out, diag


In [28]:
import time
import csv
import numpy as np

# Lists to store accuracies over epochs
global_model_weights = {}
global_model_accuracy = []
clients_train_accuracies = []  # List to store train accuracies per epoch per client
clients_test_accuracies = []   # List to store test accuracies per epoch per client

# Function to train the QNN model for one client
def train_qnn_model(client_data, client_test_data, model=None):
    if model is None:
        # Create a new QNN model if one doesn't exist
        num_features = client_data[0]["sequence"].shape[0]
        model = create_qnn_model(num_features)  # Pass num_features

    # Extract sequences and labels for training
    train_sequences = [data_point["sequence"] for data_point in client_data]
    train_labels = [data_point["label"] for data_point in client_data]

    # Extract sequences and labels for testing
    test_sequences = [data_point["sequence"] for data_point in client_test_data]
    test_labels = [data_point["label"] for data_point in client_test_data]

    # Convert lists to NumPy arrays
    train_sequences = np.array(train_sequences)
    train_labels = np.array(train_labels)
    test_sequences = np.array(test_sequences)
    test_labels = np.array(test_labels)

    print("Training started...")
    start_time = time.time()

    # Train the QNN model
    model.fit(train_sequences, train_labels)

    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Training completed in {elapsed_time} seconds.")

    # Evaluate the model on training and test data
    train_score_q = model.score(train_sequences, train_labels)
    test_score_q = model.score(test_sequences, test_labels)

    return model, train_score_q, test_score_q, elapsed_time

# Function to manually average the numerical values of the parameters across clients
def manual_average_weights(epoch_weights):
    # Initialize a list to store the summed weights (initialize with zeros)
    num_weights = len(epoch_weights[0])  # Number of weights in the model
    num_clients = len(epoch_weights)  # Number of clients

    # Initialize sum of weights to zero (assuming NumPy array or list of weights)
    summed_weights = np.zeros(num_weights)

    # Sum the weights from all clients
    for client_weights in epoch_weights:
        summed_weights += np.array(client_weights)

    # Compute the average by dividing the summed weights by the number of clients
    average_weights = summed_weights / num_clients

    return average_weights

# Function to save accuracies to CSV
def save_accuracies_to_csv(global_accuracies, clients_train_accuracies, clients_test_accuracies, filename='accuracies.csv'):
    with open(filename, mode='w', newline='') as file:
        writer = csv.writer(file)

        # Write the header row
        header = ['Epoch', 'Global Accuracy']
        for i in range(len(clients_train_accuracies[0])):  # Assuming all clients have the same number of records
            header.append(f'Client {i} Train Accuracy')
            header.append(f'Client {i} Test Accuracy')
        writer.writerow(header)

        # Write the accuracy data for each epoch
        for epoch in range(len(global_accuracies)):
            row = [epoch, global_accuracies[epoch]]  # Start with epoch and global accuracy
            for client_index in range(len(clients_train_accuracies[epoch])):
                row.append(clients_train_accuracies[epoch][client_index])  # Add train accuracy for client
                row.append(clients_test_accuracies[epoch][client_index])   # Add test accuracy for client
            writer.writerow(row)

import numpy as np

def _logits_to_labels(y_raw: np.ndarray) -> np.ndarray:
    """Robust conversion from network outputs to class labels."""
    y_raw = np.asarray(y_raw)
    if y_raw.ndim == 1:
        # Binary, single logit (e.g., expectation). Threshold at 0.
        return (y_raw >= 0).astype(int)
    elif y_raw.ndim == 2:
        if y_raw.shape[1] == 1:
            return (y_raw[:, 0] >= 0).astype(int)
        # Multi-class or 2-class with 2 outputs
        return np.argmax(y_raw, axis=1)
    else:
        raise ValueError("Unexpected network output shape: {}".format(y_raw.shape))


def compute_global_accuracy_from_weights(prototype_model, avg_weights: np.ndarray, clients: list) -> float:
    """
    Use a *fitted* prototype_model to access its underlying QNN and
    forward the averaged parameter vector 'avg_weights' over the
    concatenated global test set, returning accuracy.
    """
    # 1) Build the global test pool
    Xg = np.concatenate([np.array(item["sequence"]).reshape(1, -1) for c in clients for item in c.client_train_data[0]], axis=0)
    yg = np.concatenate([np.array(item["label"]).reshape(1,) for c in clients for item in c.client_train_data[0]], axis=0)


    # 2) Extract the underlying QNN (works for NeuralNetworkClassifier built on {Sampler,Estimator}QNN)
    qnn = getattr(prototype_model, "neural_network", None)
    if qnn is None:
        # Some versions use a protected attribute
        qnn = getattr(prototype_model, "_neural_network", None)
    if qnn is None:
        raise RuntimeError("Cannot access underlying QNN from the classifier.")

    # 3) Forward pass with the *averaged* parameter vector (no fit needed)
    y_raw = qnn.forward(Xg, np.asarray(avg_weights))

    # 4) Convert to labels and compute accuracy
    y_pred = _logits_to_labels(y_raw)
    return float(np.mean(y_pred == yg))


# Federated learning loop
num_features = 5
num_epochs = 10
global_model_weights = {}

num_clients = len(clients)
init_metrics_csv(CSV_PATH, num_clients)
global_model_accuracy = []
# Initialize 'primary_model' for each client
for client in clients:  # Assuming 'clients' is your list of client dictionaries
    client.primary_model = None  # Initialize 'primary_model' to None

for epoch in range(num_epochs):
    global_model_weights[epoch] = []
    epoch_weights = []

    epoch_train_accuracies = []  # Store train accuracies for this epoch
    epoch_test_accuracies = []   # Store test accuracies for this epoch
    print(f"Epoch: {epoch}")

    #===NEW===
    # Collect per-client stats for teleportation-aware aggregation
    epoch_weights = []
    shard_sizes = []
    latencies = []
    fidelities = []
    instabilities = []
    #===NEW===
    # Train each client
    for index, client in enumerate(clients):
        t0 = time.time()
        print(f"Training Client {index}")
        # Use dictionary key access instead of dot notation
        if client.primary_model is None:
            # Pass both training and test data to the training function
            model, train_score_q, test_score_q, train_time = train_qnn_model(client.client_train_data[epoch], np_test_data) # Use client.client_train_data and np_test_data
            client.primary_model = model
        else:
            model, train_score_q, test_score_q, train_time = train_qnn_model(client.client_train_data[epoch], np_test_data, model=client.primary_model) # Use client.client_train_data and np_test_data

        client.models.append(model)
        client.train_scores.append(train_score_q)
        client.test_scores.append(test_score_q)

        w_local = extract_client_weights(model)
        epoch_weights.append(w_local)
        shard_sizes.append(len(client.client_train_data[epoch]))

        # latency (seconds) for this client's local step
        latencies.append(max(train_time, 1e-6))

        # instability proxy
        instabilities.append(measure_instability(w_local))

        # teleportation fidelity (mean over samples) — or 1.0 if disabled
        if tele_cfg.use_teleportation:
            F_mean = float(np.mean(tele_link.sample_fidelity()))
        else:
            F_mean = 1.0
        fidelities.append(F_mean)

        print(f"Client {index} Train Score: {train_score_q}")
        print(f"Client {index} Test Score: {test_score_q}")
        print("\n\n")
        print("----------------------------------------------------------")

        # Append client accuracies for this epoch
        epoch_train_accuracies.append(train_score_q)
        epoch_test_accuracies.append(test_score_q)

        # Collect model weights (assuming model has a "weights" attribute that can be averaged)
        # REMOVE THIS DUPLICATE APPEND: epoch_weights.append(model.weights)

   # Manually compute the average weights
    #average_weights = manual_average_weights(epoch_weights)
    # Weighted sizes per client (use your own definition of |D_n| as needed)
    shard_sizes = [len(client.client_train_data[epoch]) for client in clients]

    # If you have a known angle_mask per parameter, pass it here; else None ⇒ all angular.
    #average_weights, diag = aggregate_weights_angle_aware(
        #epoch_weights,
        #shard_sizes=shard_sizes,
        #mode="auto",           # 'linear' | 'circular' | 'auto'
        #angle_mask=None        # or a boolean array of length len(epoch_weights[0])
    #)
    # === Teleportation-aware, angle-aware aggregation ===
    average_weights, diag,weights = agg_faaa_beta_with_qos(
        local_ws=epoch_weights,
        shard_sizes=shard_sizes,
        fidelities=np.asarray(fidelities),
        latencies=np.asarray(latencies),
        instabilities=np.asarray(instabilities),
        tele_cfg=tele_cfg,
        angle_mask=None  # set to boolean mask if only some coords are angles
    )



    print(f"[AUTO-FAAA] mode={diag['used_mode']} | R_mean={diag['R_mean']:.3f} | "
          f"straddle={diag['straddle_frac']:.3f} | sse_gap={diag['sse_geo_gap']:.4f} | "
          f"F̄={np.mean(fidelities):.3f}")


        # Update the global model weights
    print("Global model updated")
    # Push global weights back to all clients for warm-start next epoch
    global_model_weights[epoch] = average_weights

    for c in clients:
        if hasattr(c.primary_model, "initial_point"):
            c.primary_model.initial_point = np.asarray(average_weights)
        # Some learners also let you set .weights directly for warm-start
        # REMOVE THIS LINE: if hasattr(c.primary_model, "weights"):
            # REMOVE THIS LINE: c.primary_model.weights = np.asarray(average_weights)

    # --- insert this instead ---
    #avg_w = global_model_weights[epoch]
    # Evaluate “global” accuracy using your existing helper
    prototype = clients[0].primary_model
    global_acc = compute_global_accuracy_from_weights(prototype, average_weights, clients)
    global_model_accuracy.append(global_acc)

    # ---- save round-level telemetry
    append_round_telemetry(
        path=CSV_PATH.replace(".csv", "ICOIN_2025/Client/_round_telemetry.csv"),
        epoch=epoch,
        global_acc=global_acc,
        fidelities=np.asarray(fidelities),
        latencies=np.asarray(latencies),
        instabilities=np.asarray(instabilities),
        weights=weights,
        diag=diag
    )

    # ---- save client-level telemetry (attach trust weights to per-client rows)
    per_client_rows = []
    for cid, (tr, te, dt, F, V, sz, wloc) in enumerate(
            zip(epoch_train_accuracies, epoch_test_accuracies, latencies, fidelities, instabilities, shard_sizes, epoch_weights)):
        per_client_rows.append({
            "client_id": cid,
            "shard_size": sz,
            "train_acc_local": tr,
            "test_acc_local": te,
            "train_loss_local": 0.0,   # fill if you have them here, else 0.0
            "test_loss_local":  0.0,
            "time_sec_local": dt,
            "fidelity_mean": F,
            "instability_var": V,
            "trust_weight": float(weights[cid]),
        })

    append_client_telemetry(
        path=CSV_PATH.replace(".csv", "ICOIN_2025/CLient/_client_telemetry.csv"),
        epoch=epoch,
        per_client_rows=per_client_rows
    )


    # set starting point for the NEXT epoch’s training
    #for c in clients:
       # c.next_init = avg_w.copy() # Use dot notation
        #c.primary_model.initial_point = np.asarray(avg_w) # Use dot notation


    # # Calculate global accuracy using the test data
    # test_sequences = clients[0]["test_data"]["sequence"]  # Access test_sequences
    # test_labels = clients[0]["test_data"]["label"]  # Access test_labels
    # global_accuracy = getAccuracy(global_model_weights[epoch], num_features, test_sequences, test_labels)
    # global_model_accuracy.append(global_accuracy)

    # Save the clients' train/test accuracies for this epoch
    clients_train_accuracies.append(epoch_train_accuracies)
    clients_test_accuracies.append(epoch_test_accuracies)

    # compute global accuracy on the combined test set
    # global_acc = compute_global_accuracy(new_model_with_global_weights, clients) # This line was removed in the original cell
    # global_model_accuracy.append(global_acc) # This line was removed in the original cell
    # Example: append to a separate file
    with open("angle_diag.csv", "a", newline="") as f:
        w = csv.writer(f)
        if epoch == 0:
            w.writerow(["epoch","used_mode","R_mean","straddle_frac","sse_geo_gap"])
        w.writerow([epoch, diag["used_mode"], diag["R_mean"], diag["straddle_frac"], diag["sse_geo_gap"]])

    # APPEND one row to Drive for this epoch
    append_metrics_row(
        CSV_PATH, epoch, global_acc,
        epoch_train_accuracies, # Use the lists containing accuracies for the current epoch
        epoch_test_accuracies   # Use the lists containing accuracies for the current epoch
    )
    print(f"[Epoch {epoch}] global_acc={global_acc:.4f}")
    # print(f"Global Model Accuracy In Epoch {epoch}: {global_accuracy:.2f}")
    print("----------------------------------------------------------")
    print(f"[Epoch {epoch}] global_acc={global_acc:.4f} "
          f"| train={epoch_train_accuracies} | test={epoch_test_accuracies}")

Epoch: 0
Training Client 0
Number of features (input dimension): 5
Number of circuit parameters: 25
Training started...


/tmp/ipython-input-3714545703.py:44: DeprecationWarning: V1 Primitives are deprecated as of qiskit-machine-learning 0.8.0 and will be removed no sooner than 4 months after the release date. Use V2 primitives for continued compatibility and support.
  sampler_qnn = SamplerQNN(


Training completed in 134.59790992736816 seconds.
Client 0 Train Score: 0.87
Client 0 Test Score: 0.8248



----------------------------------------------------------
Training Client 1
Number of features (input dimension): 5
Number of circuit parameters: 25
Training started...


/tmp/ipython-input-3714545703.py:44: DeprecationWarning: V1 Primitives are deprecated as of qiskit-machine-learning 0.8.0 and will be removed no sooner than 4 months after the release date. Use V2 primitives for continued compatibility and support.
  sampler_qnn = SamplerQNN(


Training completed in 132.02119040489197 seconds.
Client 1 Train Score: 0.83
Client 1 Test Score: 0.7928



----------------------------------------------------------
Training Client 2
Number of features (input dimension): 5
Number of circuit parameters: 25
Training started...


...

Training completed in 133.75613260269165 seconds.
Client 2 Train Score: 0.87
Client 2 Test Score: 0.8168



----------------------------------------------------------
Training Client 3
Number of features (input dimension): 5
Number of circuit parameters: 25
Training started...


/tmp/ipython-input-3714545703.py:44: DeprecationWarning: V1 Primitives are deprecated as of qiskit-machine-learning 0.8.0 and will be removed no sooner than 4 months after the release date. Use V2 primitives for continued compatibility and support.
  sampler_qnn = SamplerQNN(


Training completed in 132.90233755111694 seconds.
Client 3 Train Score: 0.9
Client 3 Test Score: 0.88



----------------------------------------------------------
Training Client 4
Number of features (input dimension): 5
Number of circuit parameters: 25
Training started...


/tmp/ipython-input-3714545703.py:44: DeprecationWarning: V1 Primitives are deprecated as of qiskit-machine-learning 0.8.0 and will be removed no sooner than 4 months after the release date. Use V2 primitives for continued compatibility and support.
  sampler_qnn = SamplerQNN(


Training completed in 133.34577918052673 seconds.
Client 4 Train Score: 0.99
Client 4 Test Score: 0.9576



----------------------------------------------------------
[AUTO-FAAA] mode=circular | R_mean=0.459 | straddle=0.600 | sse_gap=8.0877 | F̄=0.951
Global model updated
[Epoch 0] global_acc=0.5740
----------------------------------------------------------
[Epoch 0] global_acc=0.5740 | train=[0.87, 0.83, 0.87, 0.9, 0.99] | test=[0.8248, 0.7928, 0.8168, 0.88, 0.9576]
Epoch: 1
Training Client 0
Training started...
Training completed in 132.77351927757263 seconds.
Client 0 Train Score: 0.78
Client 0 Test Score: 0.706



----------------------------------------------------------
Training Client 1
Training started...
Training completed in 132.78806591033936 seconds.
Client 1 Train Score: 0.78
Client 1 Test Score: 0.7154



----------------------------------------------------------
Training Client 2
Training started...
Training completed in 134.34469318389893 seconds.
Client 2 Train Score:

Visualizing Results

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Load the CSV file
filename = 'accuracies.csv'
data = pd.read_csv(filename)

# Extract the relevant columns for plotting
epochs = data['Epoch']
global_accuracy = data['Global Accuracy']
client_train_accuracies = data.filter(like='Train Accuracy').values
client_test_accuracies = data.filter(like='Test Accuracy').values

# Plot Global Accuracy over Epochs
plt.figure(figsize=(10, 6))
plt.plot(epochs, global_accuracy, label='Global Accuracy', color='blue', marker='o')
plt.xlabel('Epoch')
plt.ylabel('Global Accuracy')
plt.title('Global Accuracy Over Epochs')
plt.legend()
plt.grid(True)
plt.show()

# Plot Train Accuracies for all clients over Epochs
plt.figure(figsize=(10, 6))
for i in range(client_train_accuracies.shape[1]):
    plt.plot(epochs, client_train_accuracies[:, i], label=f'Client {i} Train Accuracy', marker='o')
plt.xlabel('Epoch')
plt.ylabel('Train Accuracy')
plt.title('Train Accuracies for All Clients Over Epochs')
plt.legend()
plt.grid(True)
plt.show()

# Plot Test Accuracies for all clients over Epochs
plt.figure(figsize=(10, 6))
for i in range(client_test_accuracies.shape[1]):
    plt.plot(epochs, client_test_accuracies[:, i], label=f'Client {i} Test Accuracy', marker='o')
plt.xlabel('Epoch')
plt.ylabel('Test Accuracy')
plt.title('Test Accuracies for All Clients Over Epochs')
plt.legend()
plt.grid(True)
plt.show()


new ways to average

In [ ]:
global_model_weights = {}
global_model_accuracy = []

# Federated learning loop
for epoch in range(num_epochs):
    global_model_weights[epoch] = []
    epoch_weights = []
    print(f"Epoch: {epoch}")

    # Train each client
    for index, client in enumerate(clients):
        print(f"Training Client {index}")

        if client.primary_model is None:
            # First time training: no existing model
            train_score_q, test_score_q, model = train(data=client.data[epoch])
            client.primary_model = model
        else:
            # Continue training with the existing model
            train_score_q, test_score_q, model = train(data=client.data[epoch], model=client.primary_model)

        # Save model and scores
        client.models.append(model)
        client.train_scores.append(train_score_q)
        client.test_scores.append(test_score_q)

        # Extract and collect model weights
        print(f"Train Score: {train_score_q}")
        print(f"Test Score: {test_score_q}")
        print("\n\n")

        # Assuming model.weights returns a NumPy array or list of weights
        epoch_weights.append(model.weights)

    # Average the weights across all clients
    average_weights = sum(epoch_weights) / len(epoch_weights)

    # Create a new model with the averaged global weights
    global_model_weights[epoch] = average_weights
    new_model_with_global_weights = create_model_with_weights(global_model_weights[epoch])

    # Update each client's primary model with the global averaged weights
    for index, client in enumerate(clients):
        client.primary_model = new_model_with_global_weights

    # Optionally calculate global accuracy (if applicable in your case)
    global_accuracy = getAccuracy(global_model_weights[epoch])  # Assuming getAccuracy() works with global weights
    global_model_accuracy.append(global_accuracy)

    print(f"Global Model Accuracy In Epoch {epoch}: {global_accuracy:.2f}")
    print("----------------------------------------------------------")


In [ ]:
# Function to extract numerical values of parameters
def extract_param_values(model):
    param_values = []
    # Loop through each parameter in the circuit and get its bound value
    for param in model.neural_network.circuit.parameters:
        # Extract the numerical value (assuming they are already bound with values)
        bound_value = model.neural_network.circuit._parameters[param]
        param_values.append(bound_value)
    return np.array(param_values)

# Function to set numerical values of parameters back into the circuit
def set_param_values(model, param_values):
    # Assign the averaged parameter values back to the circuit
    parameter_dict = {param: value for param, value in zip(model.neural_network.circuit.parameters, param_values)}
    model.neural_network.circuit.assign_parameters(parameter_dict)

# Federated learning loop
for epoch in range(num_epochs):
    global_model_weights[epoch] = []
    epoch_weights = []
    print(f"Epoch: {epoch}")

    # Train each client
    for index, client in enumerate(clients):
        print(f"Training Client {index}")

        if client.primary_model is None:
            # Pass both training and test data to the training function
            model, train_score_q, test_score_q, train_time = train_qnn_model(client.data[epoch], client.test_data[epoch])
            client.primary_model = model
        else:
            model, train_score_q, test_score_q, train_time = train_qnn_model(client.data[epoch], client.test_data[epoch], model=client.primary_model)

        client.models.append(model)
        client.train_scores.append(train_score_q)
        client.test_scores.append(test_score_q)

        print(f"Client {index} Train Score: {train_score_q}")
        print(f"Client {index} Test Score: {test_score_q}")
        print("  ")
        print("----------------------------------------------------------")

        # Collect model weights (extract numerical parameter values)
        param_values = extract_param_values(client.primary_model)
        epoch_weights.append(param_values)

    # Average the numerical values of the parameters across clients
    average_weights = np.mean(epoch_weights, axis=0)

    # Manually update each client's model parameters with the averaged weights
    global_model_weights[epoch] = average_weights

    for client in clients:
        # Manually set the global averaged weights to the model's parameters
        set_param_values(client.primary_model, global_model_weights[epoch])

    # Calculate global accuracy
    global_accuracy = getAccuracy(global_model_weights[epoch])

    # Print global accuracy for the epoch
    print(f"Global Model Accuracy In Epoch {epoch}: {global_accuracy:.2f}")
    print("----------------------------------------------------------")


33333333333333333333333333333333333333333333

In [ ]:
global_model_weights = {}
global_model_accuracy = []

# Function to train the QNN model for one client
def train_qnn_model(client_data,client_test_data, model=None):
    #num_features = client_data[0]["sequence"].shape[0]  # Get the feature dimension
    # Debug: Print the client data structure
    print("Client Test Data Structure:", client_test_data)
    if model is None:
        # Create a new QNN model if one doesn't exist
        model = create_qnn_model(client_data)

    # Extract sequences and labels for training
    train_sequences = [data_point["sequence"] for data_point in client_data]
    train_labels = [data_point["label"] for data_point in client_data]

    # Extract sequences and labels for testing
    # Handle test data
    if isinstance(client_test_data, dict):
        # Single data point (dictionary format)
        test_sequences = np.array([client_test_data["sequence"]])
        test_labels = np.array([client_test_data["label"]])
    else:
        # List of dictionaries (multiple data points)
        test_sequences = [data_point["sequence"] for data_point in client_test_data]
        test_labels = [data_point["label"] for data_point in client_test_data]
        test_sequences = np.array(test_sequences)
        test_labels = np.array(test_labels)

    # Convert lists to NumPy arrays
    train_sequences = np.array(train_sequences)
    train_labels = np.array(train_labels)


    print("Training started...")
    start_time = time.time()

    # Train the QNN model
    model.fit(train_sequences, train_labels)

    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Training completed in {elapsed_time} seconds.")

    # Evaluate the model on training and test data
    train_score_q = model.score(train_sequences, train_labels)
    test_score_q = model.score(test_sequences, test_labels)

    return model, train_score_q, test_score_q, elapsed_time

# Federated learning loop
for epoch in range(num_epochs):
    global_model_weights[epoch] = []
    epoch_weights = []
    print(f"Epoch: {epoch}")

    # Train each client
    for index, client in enumerate(clients):
        print(f"Training Client {index}")

        if client.primary_model is None:
            # Pass both training and test data to the training function
            model, train_score_q, test_score_q, train_time = train_qnn_model(client.data[epoch], client.test_data[epoch])
            client.primary_model = model
        else:
            model, train_score_q, test_score_q, train_time = train_qnn_model(client.data[epoch], client.test_data[epoch], model=client.primary_model)

        client.models.append(model)
        client.train_scores.append(train_score_q)
        client.test_scores.append(test_score_q)

        print(f"Client {index} Train Score: {train_score_q}")
        print(f"Client {index} Test Score: {test_score_q}")
        print("  ")
        print("----------------------------------------------------------")

        for model in client.models:
        # Extract numerical values of parameters (as a NumPy array)
            param_values = [p.value for p in model.neural_network.circuit.parameters] # Extract the values
            epoch_weights.append(param_values)

    # Average the numerical values of the parameters across clients
    average_weights = np.mean(epoch_weights, axis=0)

    # Manually update each client's model parameters with the averaged weights
    global_model_weights[epoch] = average_weights

    for client in clients:
        # Manually set the global averaged weights to the model's parameters
        for i, param in enumerate(client.primary_model.neural_network.circuit.parameters):
            client.primary_model.neural_network.circuit._parameters[param] = average_weights[i] # Set the value directly


    # Calculate global accuracy
    global_accuracy = getAccuracy(global_model_weights[epoch])

    # Print global accuracy for the epoch
    print(f"Global Model Accuracy In Epoch {epoch}: {global_accuracy:.2f}")
    print("----------------------------------------------------------")


In [ ]:

import time

itr = 0
def training_callback(weights, obj_func_eval):
        global itr
        itr += 1
        print(f"{itr}", end=' | ')
def train(data, model = None):
  if model is None:
    num_features = len(data[0]["sequence"])
    feature_map = ZZFeatureMap(feature_dimension=num_features, reps=1)
    ansatz = RealAmplitudes(num_qubits=num_features, reps=3)
    optimizer = COBYLA(maxiter=max_train_iterations)
    vqc_model = VQC(
        feature_map=feature_map,
        ansatz=ansatz,
        optimizer=optimizer,
        callback=partial(training_callback),
        sampler=BackendSampler(backend=backend),
        warm_start=True
    )
    model = vqc_model

  train_sequences = [data_point["sequence"] for data_point in data]
  train_labels = [data_point["label"] for data_point in data]

  # Convert the lists to NumPy arrays
  train_sequences = np.array(train_sequences)
  train_labels = np.array(train_labels)

  # Print the shapes
  print("Train Sequences Shape:", train_sequences.shape)
  print("Train Labels Shape:", train_labels.shape)

  print("Training Started")
  start_time = time.time()
  model.fit(train_sequences, train_labels)
  end_time = time.time()
  elapsed_time = end_time - start_time
  print(f"\nTraining complete. Time taken: {elapsed_time} seconds.")

  print(f"SCORING MODEL")
  train_score_q = model.score(train_sequences, train_labels)
  test_score_q = model.score(test_sequences[:200], test_labels[:200])
  return train_score_q, test_score_q, model

def getAccuracy(weights):
        num_features = len(test_sequences[0])
        feature_map = ZZFeatureMap(feature_dimension=num_features, reps=1)
        ansatz = RealAmplitudes(num_qubits=num_features, reps=3)
        ansatz = ansatz.bind_parameters(weights)
        optimizer = COBYLA(maxiter=0)
        vqc = VQC(
            feature_map=feature_map,
            ansatz=ansatz,
            optimizer=optimizer,
            sampler=BackendSampler(backend=backend)
        )
        vqc.fit(test_sequences[:25], test_labels[:25])
        return vqc.score(test_sequences[:200], test_labels[:200])

def create_model_with_weights(weights):
  num_features = len(test_sequences[0])
  feature_map = ZZFeatureMap(feature_dimension=num_features, reps=1)
  ansatz = RealAmplitudes(num_qubits=num_features, reps=3)
  optimizer = COBYLA(maxiter=max_train_iterations)
  vqc = VQC(
      feature_map=feature_map,
      ansatz=ansatz,
      optimizer=optimizer,
      sampler=BackendSampler(backend=backend),
      warm_start = True,
      initial_point  = weights,
      callback=partial(training_callback)
  )
  return vqc


global_model_weights = {}
global_model_accuracy = []

for epoch in range(num_epochs):
  global_model_weights[epoch] = []
  epoch_weights = []
  print(f"epoch: {epoch}")
  for index, client in enumerate(clients):
    print(f"Index: {index}, Client: {client}")

    if client.primary_model is None:
      train_score_q, test_score_q, model = train(data = client.data[epoch])
      client.models.append(model)
      client.test_scores.append(test_score_q)
      client.train_scores.append(train_score_q)
      # Print the values
      print("Train Score:", train_score_q)
      print("Test Score:", test_score_q)
      print("\n\n")
      epoch_weights.append(model.weights)

    else:
      train_score_q, test_score_q, model = train(data = client.data[epoch], model = client.primary_model)
      client.models.append(model)
      client.test_scores.append(test_score_q)
      client.train_scores.append(train_score_q)
      print("Train Score:", train_score_q)
      print("Test Score:", test_score_q)
      print("\n\n")
      epoch_weights.append(model.weights)


if(epoch != 0):
    epoch_weights.append(global_model_weights[epoch-1])
average_weights = sum(epoch_weights) / len(epoch_weights)

global_model_weights[epoch] = average_weights
new_model_with_global_weights = create_model_with_weights(global_model_weights[epoch])
for index, client in enumerate(clients):
  client.primary_model = new_model_with_global_weights

global_accuracy = getAccuracy(global_model_weights[epoch])
print(f"Global Model Accuracy In Epoch {epoch}: {global_accuracy}")
print("----------------------------------------------------------")
global_model_accuracy.append(global_accuracy)

